In [1]:
# استيراد الدالة من السكريبت
from match_bert_embeddings import match_bert_embeddings

# استدعاء الدالة مباشرة
match_bert_embeddings(
    docs_embedding_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject/data/doc_bert_embeddings.joblib",
    queries_embedding_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject/data/query_bert_embeddings.joblib",
    output_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject/results/bert_results.json",
    top_k=100,
    batch_size_queries=100
)


In [3]:
if __name__ == "__main__":
    docs_embedding_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\Bert\beir\quora\test\doc\bert_embedding.joblib"
    queries_embedding_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Query Processing\Bertquery\BEIR\quora\test\query_embeddings\bert_query_embeddings.joblib"
    output_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Query Matching & Ranking\BertMatching\bert_results.json"

    match_bert_embeddings(
        docs_embedding_path,
        queries_embedding_path,
        output_path,
        top_k=100,
        batch_size_queries=100
    )


📥 تحميل تمثيلات الوثائق...
📥 تحميل تمثيلات الاستعلامات...
📊 عدد الاستعلامات: 10000, عدد الوثائق: 522931


🔄 مطابقة دفعات الاستعلامات: 100%|██████████| 100/100 [03:28<00:00,  2.09s/it]


✅ تم حفظ النتائج في: C:\Users\Azzam\PycharmProjects\PythonProject\Query Matching & Ranking\bert_results.json


In [5]:
import joblib
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import json
import os

def match_bert_embeddings(
    docs_embedding_path: str,
    queries_embedding_path: str,
    output_path: str = "bert_antique_results.json",
    top_k: int = 100,
    batch_size_queries: int = 100
):
    print("📥 تحميل تمثيلات الوثائق...")
    doc_data = joblib.load(docs_embedding_path)
    doc_ids = doc_data["doc_ids"]
    doc_embeddings = np.vstack(doc_data["embeddings_matrix"])  # shape: (num_docs, dim)

    print("📥 تحميل تمثيلات الاستعلامات...")
    query_data = joblib.load(queries_embedding_path)
    query_ids = query_data["query_ids"]
    query_embeddings = np.vstack(query_data["embeddings"])  # shape: (num_queries, dim)

    results = {}

    num_queries = len(query_embeddings)
    print(f"📊 عدد الاستعلامات: {num_queries}, عدد الوثائق: {len(doc_ids)}")

    for start in tqdm(range(0, num_queries, batch_size_queries), desc="🔄 مطابقة دفعات الاستعلامات"):
        end = min(start + batch_size_queries, num_queries)
        batch_queries = query_embeddings[start:end]

        sim_matrix = cosine_similarity(batch_queries, doc_embeddings)

        for i, query_idx in enumerate(range(start, end)):
            sims = sim_matrix[i]
            top_indices = np.argpartition(sims, -top_k)[-top_k:]
            top_scores = sims[top_indices]
            sorted_idx = top_indices[np.argsort(-top_scores)]

            # ✅ تعديل هنا لتخزين القيم كمصفوفات بدل tuple
            results[str(query_ids[query_idx])] = [
                [str(doc_ids[idx]), float(sims[idx])] for idx in sorted_idx
            ]

    # حفظ النتائج
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"✅ تم حفظ النتائج في: {output_path}")




In [7]:
if __name__ == "__main__":
    docs_embedding_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\Bert\antique\train\doc\bert_embedding.joblib"
    queries_embedding_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Query Processing\Bertquery\antique\train\query_embeddings\bert_query_embeddings.joblib"
    output_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Query Matching & Ranking\BertMatching\bert_antique_results.json"

    match_bert_embeddings(
        docs_embedding_path,
        queries_embedding_path,
        output_path,
        top_k=100,
        batch_size_queries=100
    )


📥 تحميل تمثيلات الوثائق...
📥 تحميل تمثيلات الاستعلامات...
📊 عدد الاستعلامات: 176, عدد الوثائق: 401768


🔄 مطابقة دفعات الاستعلامات: 100%|██████████| 2/2 [00:02<00:00,  1.25s/it]


✅ تم حفظ النتائج في: C:\Users\Azzam\PycharmProjects\PythonProject\Query Matching & Ranking\BertMatching\bert_antique_results.json
